# Quantum Mechanics: Finite Difference Diagonalization
## Introduction
The key challenge in solving many quantum systems is to find the energy eigenstates for a given potential by solving the time-independent Schrodinger equation:
$$ H\psi_n = \left(\frac{p^2}{2m}+V\right)\psi_n=E_n\psi$$
When we work in real space, this equation becomes:
In real space, it acts on the wavefunction as:
$$ H\psi_n(x) = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2}\psi(x) + V(x)\psi_n(x)=E_n\psi_n(x)$$
This differential equation can be quite difficult to solve, and analytic solutions are only available for a few well known systems, like the Infinite Square Well and the Quantum Harmonic Oscillator.  Thus, physicists need to use computers and numerical algorithms in order to solve Schrodinger's equation for most physical systems.  In this notebook you'll learn about one particularly useful (and surprsingly simple!) method to find stationary states for arbitrary 1D potentials - Finite Difference Diagonalization.

## Finite Differences
The time independent Schrodinger equation involves derivatives of the wavefunction, so a first task is to figure out how to calcualte derivatives of functions on a computer.  One way of doing this is via Finite Differences, which is actually just a fancy way of describing how you learned to do derivatives in Introductory Calculus.  Back then, you learned that the derivative can be defined as:
$$ \frac{df}{dx} = \lim_{\Delta x \rightarrow 0}\frac{f(x+\Delta x)-f(x)}{\Delta x}$$
On a computer then, we can get an approximation for the derivative by dropping the limit and simply using a small $\Delta x$.
$$ \frac{df}{dx} \approx \frac{f(x+\Delta x)-f(x)}{\Delta x}$$
This is called the Forward Difference Approximation to the derivative.  We could also use the Backward Difference Approximation:
$$ \frac{df}{dx} \approx \frac{f(x)-f(x-\Delta x)}{\Delta x}$$
But how do we approximation a second derivative, as appears in the Schrodinger Equation?  One method is to use the Forward Difference Approximation to calculate the first derivative, and use the Backward Difference Appoximation to get the second derivative:
$$\frac{d^2f}{dx^2}\approx \frac{d}{dx}\frac{f(x+\Delta x)-f(x)}{\Delta x} \approx \frac{f(x+\Delta x)-2f(x)+f(x-\Delta)}{\Delta x^2}$$
These approximations to the derivatives should be fairly accurate as long as $\Delta x$ is small.  How small? Usually we want $\Delta x$ to be small compared to the distance over which the function varies a lot.  So for a very smooth and gently sloping curve you might be able to use a fairly large $\Delta x$, whereas you might need a much smaller one for functions that fluctuate sharply.

## Discretizing the Schrodinger Equation
We can't represent a continuous function on a computer, like $f(x)$.  Instead, we can only record the value of the function at a finite number of positions, and an intuitive choice is to save the values of the function on a regular grid of spatial locations, 
$$f(x)\rightarrow f(x_i)$$
$$x_i = n\cdot \Delta x$$
If we discretize space for the Schrodinger Equation by only recording the values of the wavefunction on a regular grid in space, we have:
$$ H\psi_n(x_i) = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2}\psi(x) + V(x)\psi_n(x)=E_n\psi_n(x_i)$$
Now if we apply our finite difference approximation for the second derivative, we get a finite difference version of the Time Tndependent Schrodinger Tquation (TISE). Setting $\hbar=m=1$, we have:
$$ H\psi_(x_i) = -\frac{1}{2}\frac{\psi(x_i+\Delta x)-2\psi(x_i) + \psi(x_i-\Delta x)}{\Delta x^2} + V(x_i)\psi_n(x_i) $$
Or, since $x_{i+1}=x_i+\Delta x$, we have:
$$ H\psi(x_i) = -\frac{1}{2}\frac{\psi(x_{i+1})-2\psi(x_i) + \psi(x_{i-1})}{\Delta x^2} + V(x_i)\psi(x_i) $$

## Matrix Diagonalization
If we think of the values of our wavefunction as the elements of a vector:
$$\vec{\psi}=\begin{bmatrix} \psi(x_0) \\ \psi(x_1) \\ ... \\ \psi(x_{n-1})\end{bmatrix}$$
We can view the finite difference TISE as a matrix equation:
$$H\vec{\psi}=-\frac{1}{2}\frac{1}{\Delta x^2}\begin{bmatrix}-2 & 1 & 0 & ...\\1 & -2 & 1 & ... \\ & & ... & \\ ... & 0 & 1 & -2 \end{bmatrix} \begin{bmatrix} \psi(x_0) \\ \psi(x_1) \\ ... \\ \psi(x_{n-1})\end{bmatrix} + \begin{bmatrix}V(x_0) &  0 & 0 & ...\\0 & V(x_1) & 0 & ... \\ & & ... & \\ ... & 0 & 0 & V(x_{n-1}) \end{bmatrix} \begin{bmatrix} \psi(x_0) \\ \psi(x_1) \\ ... \\ \psi(x_{n-1})\end{bmatrix}$$
This is remarkable, because it means that we have replaced the original differential equation with a much simpler matrix equation for the TISE:
$$H\vec{\psi}_n=E_n \vec{\psi}_n $$
But this is just a matrix eigenvalue problem! So, if we want to find the stationary states for the Hamiltonian, then we just need to find the eigenenergies and eigenstates (vectors) for the matrix $H$.  This would be hard to do by hand, but as we'll see diagonalizing even very large matrices is trivial on a computer.  The energies and eigenstates we get will depend on our discretization size - we expect to get more accurate answers the smaller we make $\Delta x$. 

## System Size
One thing you might have noticed is that in order to have a finite grid, we need to do more than just discretize space. We also need to make the space finite in extent - that is, instead of dealing with $x=-\infty$ to $x=\infty$, we deal with a discretized space from $x_0$ to $x_{n-1}$. But how big of a space do we need in order to get accurate answers?

The quick answer is that the discretized grid needs to include all positions where the wavefunction has a significant amplitude.  This means that the algorithm will be limited to bound state problems - if the particle is not bound, then the wavefunction oscillates to infinity, and so cutting space off at any distance will produce significant errors. 

It also means that the size of the space needed will depend one which eigenstates we want to obtain accurately.  Larger energy states tend to take up more space and oscillate more quickly, so we'll need larger grids with finer discretization to compute them accurately.  This is also apparent from the matrix equation we derived - if the space discretization is such that our wavefunction vector has $N$ elements, then the diagonalization of the matrix equation will yield $N$ eigenvectors.  For many bound state problems, such as the Quantum Harmonic Oscillator, the actual number of eigenstates is infinite, so we expect our algorithm to cut off the higher energy states.